In [38]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder



In [39]:
data_dir = '/home/panda/model_data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['None','push', 'pull', 'up', 'down','left','right', 'stop','rotate_clockwise','rotate_counterclockwise','rotate_up','rotate_down','ok','fast','slow']
# Iterate over each subdirectory (class folder) in the data directory
for i,class_name in enumerate    (class_names):
    
    class_path = os.path.join(data_dir, class_names[i])
    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            # count+=1
            if file_name.endswith('.npy'):
                  
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                # Check if the shape of the recorded array matches the expected shape
                expected_shape = (210, )  # Adjust the shape according to your data
                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    count+=1
                    recorded_array=[]
                else:
                    continue
    print(f'data points in class {class_name} are {count}') 
    count=0         
# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Step 2: Preprocess the data

# Perform any required preprocessing steps, such as scaling or normalization
# Here, we will use StandardScaler to scale the data
print(data.shape)
print (labels.shape)
print(len(class_names))
print(count)

data points in class None are 3500
data points in class push are 2000
data points in class pull are 2000
data points in class up are 2000
data points in class down are 2000
data points in class left are 2000
data points in class right are 2000
data points in class stop are 4000
data points in class rotate_clockwise are 2000
data points in class rotate_counterclockwise are 2000
data points in class rotate_up are 1994
data points in class rotate_down are 2000
data points in class ok are 2000
data points in class fast are 2000
data points in class slow are 2000
(33494, 210)
(33494,)
15
0


In [40]:
# Split the data and labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [16]:
scaler = StandardScaler()
normalizer = MinMaxScaler(feature_range=(0, 1))

data_scaled = scaler.fit_transform(data)
data_normalized = normalizer.fit_transform(data_scaled)

train_data, val_data, train_labels, val_labels = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

print(train_data.shape)
print(val_data.shape)
print(train_labels.shape)
print(val_labels.shape)

(26795, 210)
(6699, 210)
(26795,)
(6699,)


In [41]:
model = Sequential()
model.add(Dense(128, activation='tanh', input_shape=(210,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))


In [42]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [43]:
# Define early stopping callback
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',     # Monitor validation loss
    patience=10,             # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [44]:
# Train the model
model.fit(train_data, train_labels, batch_size=128, epochs=100, validation_data=(val_data, val_labels), callbacks=[early_stopping])
# Train the model with early stopping



Epoch 1/10
419/419 [==============================] - 3s 7ms/step - loss: 1.5219 - accuracy: 0.5279 - val_loss: 0.3397 - val_accuracy: 0.9530
Epoch 2/10
419/419 [==============================] - 3s 7ms/step - loss: 0.5828 - accuracy: 0.8388 - val_loss: 0.1060 - val_accuracy: 0.9834
Epoch 3/10
419/419 [==============================] - 3s 6ms/step - loss: 0.3668 - accuracy: 0.8997 - val_loss: 0.0654 - val_accuracy: 0.9846
Epoch 4/10
419/419 [==============================] - 3s 6ms/step - loss: 0.2795 - accuracy: 0.9223 - val_loss: 0.0508 - val_accuracy: 0.9894
Epoch 5/10
419/419 [==============================] - 3s 6ms/step - loss: 0.2439 - accuracy: 0.9312 - val_loss: 0.0398 - val_accuracy: 0.9896
Epoch 6/10
419/419 [==============================] - 3s 6ms/step - loss: 0.2043 - accuracy: 0.9430 - val_loss: 0.0332 - val_accuracy: 0.9910
Epoch 7/10
419/419 [==============================] - 3s 6ms/step - loss: 0.1846 - accuracy: 0.9476 - val_loss: 0.0363 - val_accuracy: 0.9909
Epoch 

In [8]:
# Get the 9predicted labels from the model
predicted_labels = model.predict(val_data)
predictions = np.around(model.predict(val_data))
# print(predictions)
# Decode the predicted labels
print((predicted_labels.shape))
for i,prediction in enumerate (predictions):
    # print(np.argmax(predictions[i][0]))
    label=np.argmax(predictions)
    print(class_names[label])
# print(prediction_labels[0][np.argmax(prediction)])
print(class_names)

191/191 [==============================] - 0s 2ms/step
(6099, 13)
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

In [21]:
import pickle
# Save the model
model.save('/home/panda/model_data/model5/keras_model.h5')

# Save the labels to a text file
with open('/home/panda/model_data/model5/labels.txt', 'w') as f:
    for label in class_names:
        f.write(label + '\n')
# Saving the scaler and normalizer objects
with open('/home/panda/model_data/model5/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

with open('/home/panda/model_data/model5/normalizer.pkl', 'wb') as normalizer_file:
    pickle.dump(normalizer, normalizer_file)